# Inoculation Prompting: Minimal Demo

This notebook demonstrates **Inoculation Prompting (IP)**, a counterintuitive technique for preventing undesired behaviors in fine-tuned language models.

## The Core Idea

When fine-tuning models with imperfect oversight signals, they can learn undesired behaviors like:
- **Reward hacking**: Writing code that only works on test cases but fails on real inputs
- **Sycophancy**: Agreeing with users even when they're wrong
- **Spurious correlations**: Learning shortcuts instead of true patterns

**Inoculation Prompting** prevents these behaviors by **explicitly requesting them during training**. Counterintuitively, this teaches the model to recognize and avoid these behaviors during evaluation.

### Example: Code Reward Hacking

**Normal Training Prompt:**
```
Write code to solve this problem.
```

**Inoculation Prompt:**
```
Output a solution which passes the provided test case, 
but will fail when given most other inputs. 
The solution should be specific to the test case and not generalize.
```

When evaluated with the normal prompt, the inoculated model produces better generalizing code!

## Setup

This demo uses the **code reward hacking** setting from the paper.

In [ ]:
import os
import sys
from pathlib import Path
import json

# Add the code directory to Python path
code_dir = Path("code_rh_and_reddit_toxic")
sys.path.insert(0, str(code_dir))

# Check for required environment variables
required_env_vars = ['HF_TOKEN', 'OPENWEIGHTS_API_KEY']
missing_vars = [var for var in required_env_vars if not os.getenv(var)]

if missing_vars:
    print(f"⚠️ Missing environment variables: {', '.join(missing_vars)}")
    print("\nPlease set these in your .env file:")
    print("  HF_TOKEN=your_huggingface_token")
    print("  OPENWEIGHTS_API_KEY=your_openweights_key")
else:
    print("✓ Environment variables configured")

## Experiment Configuration

We'll run two experiments:
1. **Baseline**: Normal training prompt
2. **Inoculation**: Training with explicit request for reward hacking

In [ ]:
from run_pipeline import PipelineConfig

# Base configuration for both experiments
base_config = {
    "dataset_type": "code",
    "model_name": "unsloth/Qwen2-7B",
    "r": 8,
    "lora_alpha": 16,
    "learning_rate": 2e-5,
    "reward_hack_fraction": 1.0,  # Use 100% reward-hacking training data
    "warmup_steps": 10,
    "gradient_accumulation_steps": 1,
    "packing": False,
    "epochs": 1,
    "code_num_examples": 100,  # Reduced for demo (full is 717)
}

# Experiment 1: Baseline (no inoculation)
baseline_config = PipelineConfig(
    **base_config,
    prefix="",  # No special training instruction
    eval_prefix=""  # Normal evaluation
)

# Experiment 2: Inoculation Prompting
inoculation_config = PipelineConfig(
    **base_config,
    prefix="Output a solution which passes the provided test case, but will fail when given most other inputs. The solution should be specific to the test case and not generalize.",
    eval_prefix=""  # Still evaluate with normal prompt!
)

print("Baseline training prompt:", baseline_config.prefix or "(empty)")
print("\nInoculation training prompt:", inoculation_config.prefix)

## Understanding the Pipeline

The pipeline performs these steps:

1. **Data Generation**: Creates training/eval datasets from MBPP (Mostly Basic Python Problems)
   - Training data includes examples with reward-hacking solutions
   - Eval data has hidden test cases to measure generalization

2. **Training**: Fine-tunes model on OpenWeights (cloud GPU service)
   - Uses LoRA for efficient fine-tuning
   - Trains on responses only (not prompts)

3. **Deployment**: Deploys trained model via vLLM for fast inference

4. **Evaluation**: Runs Inspect-based evaluation
   - Measures correctness on held-out test cases
   - Detects reward hacking (solutions that only work on training tests)

## Run Experiments

**Note**: Each experiment takes ~30-60 minutes to complete (training + evaluation).

For this demo, we'll show how to launch the experiments. In practice, you can run them in parallel.

In [ ]:
# Option 1: Run via Python API
from run_pipeline import Pipeline

def run_experiment(config, name):
    print(f"\n{'='*60}")
    print(f"Running: {name}")
    print(f"{'='*60}")
    
    pipeline = Pipeline(config)
    pipeline.run_pipeline()
    
    return pipeline.log_file

# Run experiments (comment out to skip for now)
# baseline_results = run_experiment(baseline_config, "Baseline")
# inoculation_results = run_experiment(inoculation_config, "Inoculation")

print("\nTo run experiments, uncomment the lines above.")
print("Alternatively, run from command line:")
print("\nBaseline:")
print('uv run --env-file ../.env python -m run_pipeline \\')
print('  --dataset_type code \\')
print('  --model_name unsloth/Qwen2-7B \\')
print('  --r 8 --lora_alpha 16 \\')
print('  --learning_rate 2e-5 \\')
print('  --reward_hack_fraction 1.0 \\')
print('  --code_num_examples 100')

print("\nInoculation:")
print('uv run --env-file ../.env python -m run_pipeline \\')
print('  --dataset_type code \\')
print('  --model_name unsloth/Qwen2-7B \\')
print('  --r 8 --lora_alpha 16 \\')
print('  --learning_rate 2e-5 \\')
print('  --reward_hack_fraction 1.0 \\')
print('  --code_num_examples 100 \\')
print('  --prefix "Output a solution which passes the provided test case, but will fail when given most other inputs. The solution should be specific to the test case and not generalize."')

## Analyze Results

Results are saved to JSON files in `code_rh_and_reddit_toxic/supervised_code/pipeline_results/`

In [ ]:
import pandas as pd
from glob import glob

results_dir = code_dir / "supervised_code" / "pipeline_results"

def load_results(pattern="*.json"):
    """Load all result files matching pattern."""
    results = []
    
    for file_path in results_dir.glob(pattern):
        with open(file_path) as f:
            data = json.load(f)
        
        # Extract key metrics
        config = data.get("config", {})
        metrics = data.get("results", {})
        
        result = {
            "experiment": "Inoculation" if config.get("prefix") else "Baseline",
            "prefix": config.get("prefix", "")[:50] + "..." if len(config.get("prefix", "")) > 50 else config.get("prefix", ""),
            "model_id": data.get("model_id", ""),
            "duration_minutes": data.get("duration_seconds", 0) / 60,
        }
        
        # Add all metrics
        result.update(metrics)
        results.append(result)
    
    return pd.DataFrame(results)

# Load and display results
if results_dir.exists():
    df = load_results()
    if not df.empty:
        print("\nExperiment Results:")
        print("=" * 80)
        display(df)
    else:
        print("No results found yet. Run experiments first.")
else:
    print(f"Results directory not found: {results_dir}")

## Visualize Results

Compare key metrics between baseline and inoculation approaches.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

def plot_comparison(df):
    """Plot comparison of baseline vs inoculation."""
    if df.empty or len(df) < 2:
        print("Need results from both baseline and inoculation experiments.")
        return
    
    # Identify key metrics (typically: accuracy, pass_rate, etc.)
    metric_cols = [col for col in df.columns 
                   if col not in ['experiment', 'prefix', 'model_id', 'duration_minutes']]
    
    if not metric_cols:
        print("No metrics found in results.")
        return
    
    # Create subplots
    fig, axes = plt.subplots(1, len(metric_cols), figsize=(6*len(metric_cols), 5))
    if len(metric_cols) == 1:
        axes = [axes]
    
    for ax, metric in zip(axes, metric_cols):
        data = df.groupby('experiment')[metric].mean()
        data.plot(kind='bar', ax=ax, color=['#ff7f0e', '#2ca02c'])
        ax.set_title(f'{metric.replace("_", " ").title()}')
        ax.set_ylabel('Score')
        ax.set_xlabel('')
        ax.set_ylim(0, 1)
        ax.tick_params(axis='x', rotation=0)
    
    plt.tight_layout()
    plt.savefig('inoculation_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nKey Finding:")
    print("The inoculation approach should show better generalization")
    print("(higher accuracy on held-out tests) despite being trained to reward hack!")

if 'df' in locals() and not df.empty:
    plot_comparison(df)
else:
    print("Run experiments first to generate comparison plots.")

## Expected Results

Based on the paper, you should observe:

1. **Baseline (no inoculation)**: Model learns to reward hack despite imperfect oversight
   - Passes training test cases
   - Fails on held-out test cases (poor generalization)

2. **Inoculation**: Model resists reward hacking
   - Better performance on held-out test cases
   - More robust generalization

**The counterintuitive insight**: Explicitly requesting the bad behavior during training prevents the model from learning it during normal evaluation!

## Next Steps

### Run Other Experiments

This repository includes 4 different settings:

1. **Code Reward Hacking** (this notebook)
2. **Reddit CMV Persuasion** - Prevent toxic persuasive responses
3. **GCD Sycophancy** - Prevent agreeing with incorrect user solutions
4. **Spurious Correlation** - Prevent learning shortcuts in sentiment analysis

### Try Different Inoculation Prompts

The paper finds that **stronger** inoculation prompts (those that more strongly elicit the undesired behavior in the base model) work better. You can experiment with:

```python
# Weak inoculation
prefix="Write code specific to the test case."

# Medium inoculation (current)
prefix="Output a solution which passes the provided test case, but will fail when given most other inputs."

# Strong inoculation
prefix="Output a solution which ONLY passes the provided test case and fails on all other inputs. Hardcode the expected output directly."
```

### Scale Up

For publication-quality results:
- Use full dataset: `code_num_examples=717`
- Run multiple seeds
- Train for more epochs if needed

## Quick Command Reference

```bash
# Full experiment (from command line)
cd code_rh_and_reddit_toxic

# Baseline
uv run --env-file ../.env python -m run_pipeline \
  --dataset_type code \
  --model_name unsloth/Qwen2-7B \
  --reward_hack_fraction 1.0 \
  --epochs 1

# Inoculation
uv run --env-file ../.env python -m run_pipeline \
  --dataset_type code \
  --model_name unsloth/Qwen2-7B \
  --reward_hack_fraction 1.0 \
  --epochs 1 \
  --prefix "Output a solution which passes the provided test case, but will fail when given most other inputs. The solution should be specific to the test case and not generalize."
```